# 🧹 Salifort Motors Employee Retention Analysis
## Phase 2: Data Cleaning and Preprocessing (PACE - Analyze)

**Project Overview:** This notebook focuses on cleaning and preprocessing our employee data to ensure high-quality input for our machine learning models.

**Objectives:**
- Handle missing values and outliers
- Clean and standardize data formats
- Create analysis-ready datasets
- Prepare features for modeling

---

### 📋 Table of Contents
1. [Setup & Data Loading](#setup)
2. [Data Quality Assessment](#quality)
3. [Missing Values Treatment](#missing)
4. [Outlier Detection & Treatment](#outliers)
5. [Data Standardization](#standardization)
6. [Feature Engineering](#features)
7. [Final Dataset Export](#export)

---

## 🛠️ Setup & Data Loading {#setup}

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print("📊 Ready for data cleaning and preprocessing")

In [ ]:
# Load the raw dataset
try:
    df_raw = pd.read_csv('../data/raw/hr_dataset.csv')
    print(f"✅ Raw dataset loaded successfully!")
    print(f"📏 Original shape: {df_raw.shape}")
    
    # Create a working copy
    df = df_raw.copy()
    print(f"📋 Working copy created for data cleaning")
    
except FileNotFoundError:
    print("❌ Dataset file not found. Please ensure the file is in the correct location.")
    print("Expected location: ../data/raw/hr_dataset.csv")

## 🔍 Data Quality Assessment {#quality}

In [ ]:
# Comprehensive data quality assessment
print("🔍 COMPREHENSIVE DATA QUALITY ASSESSMENT")
print("=" * 60)

quality_report = {
    'Total Records': df.shape[0],
    'Total Features': df.shape[1],
    'Memory Usage (MB)': round(df.memory_usage(deep=True).sum() / 1024**2, 2)
}

print("📊 BASIC STATISTICS:")
for key, value in quality_report.items():
    print(f"  {key}: {value:,}")

print("\n📋 COLUMN INFORMATION:")
print("-" * 60)
print(f"{'Column Name':<25} {'Type':<15} {'Missing':<10} {'Unique':<10}")
print("-" * 60)

for col in df.columns:
    missing_count = df[col].isnull().sum()
    missing_pct = f"{(missing_count/len(df)*100):.1f}%"
    unique_count = df[col].nunique()
    dtype = str(df[col].dtype)
    
    print(f"{col:<25} {dtype:<15} {missing_pct:<10} {unique_count:<10}")

print("\n" + "=" * 60)

In [ ]:
# Check for duplicate records
print("🔄 DUPLICATE RECORDS CHECK")
print("=" * 50)

duplicate_count = df.duplicated().sum()
duplicate_pct = (duplicate_count / len(df)) * 100

print(f"Total duplicate records: {duplicate_count:,} ({duplicate_pct:.2f}%)")

if duplicate_count > 0:
    print("\n🔍 Sample duplicate records:")
    duplicates = df[df.duplicated(keep=False)].sort_values(list(df.columns))
    display(duplicates.head(10))
    
    # Remove duplicates
    df_before = df.shape[0]
    df = df.drop_duplicates()
    df_after = df.shape[0]
    
    print(f"\n✅ Duplicates removed: {df_before - df_after:,} records")
    print(f"📏 New dataset shape: {df.shape}")
else:
    print("✅ No duplicate records found!")

## 🩹 Missing Values Treatment {#missing}

In [ ]:
# Detailed missing values analysis
print("🩹 MISSING VALUES ANALYSIS & TREATMENT")
print("=" * 50)

missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing_data.index,
    'Missing_Count': missing_data.values,
    'Missing_Percentage': missing_percent.values
}).sort_values('Missing_Count', ascending=False)

# Display missing values summary
missing_summary = missing_df[missing_df['Missing_Count'] > 0]

if len(missing_summary) > 0:
    print("📊 Columns with missing values:")
    display(missing_summary)
    
    # Visualize missing data pattern
    if len(missing_summary) > 0:
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        sns.barplot(data=missing_summary, y='Column', x='Missing_Count')
        plt.title('Missing Values Count by Column')
        plt.xlabel('Number of Missing Values')
        
        plt.subplot(1, 2, 2)
        sns.barplot(data=missing_summary, y='Column', x='Missing_Percentage')
        plt.title('Missing Values Percentage by Column')
        plt.xlabel('Percentage of Missing Values')
        
        plt.tight_layout()
        plt.show()
    
    # Handle missing values based on column type and percentage
    print("\n🔧 Missing values treatment strategy:")
    
    for _, row in missing_summary.iterrows():
        col = row['Column']
        missing_pct = row['Missing_Percentage']
        
        if missing_pct > 50:
            print(f"  {col}: DROP COLUMN (>{missing_pct:.1f}% missing)")
            df = df.drop(columns=[col])
        elif df[col].dtype in ['object']:
            mode_val = df[col].mode()[0] if len(df[col].mode()) > 0 else 'Unknown'
            print(f"  {col}: FILL with mode '{mode_val}' ({missing_pct:.1f}% missing)")
            df[col].fillna(mode_val, inplace=True)
        else:
            median_val = df[col].median()
            print(f"  {col}: FILL with median {median_val} ({missing_pct:.1f}% missing)")
            df[col].fillna(median_val, inplace=True)
    
    print(f"\n✅ Missing values treatment completed!")
    print(f"📏 Final dataset shape: {df.shape}")
    
else:
    print("✅ No missing values found in the dataset!")

# Verify no missing values remain
remaining_missing = df.isnull().sum().sum()
print(f"\n🔍 Remaining missing values: {remaining_missing}")

## 🎯 Outlier Detection & Treatment {#outliers}

In [ ]:
# Outlier detection for numerical columns
print("🎯 OUTLIER DETECTION & ANALYSIS")
print("=" * 50)

numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Remove target variable from outlier analysis if it exists
target_col = 'left'  # Update if your target has a different name
if target_col in numerical_cols:
    numerical_cols.remove(target_col)

print(f"📊 Analyzing {len(numerical_cols)} numerical columns for outliers...")

outlier_summary = []

# Function to detect outliers using IQR method
def detect_outliers_iqr(column_data):
    Q1 = column_data.quantile(0.25)
    Q3 = column_data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = column_data[(column_data < lower_bound) | (column_data > upper_bound)]
    return outliers, lower_bound, upper_bound

# Detect outliers for each numerical column
for col in numerical_cols:
    outliers, lower_bound, upper_bound = detect_outliers_iqr(df[col])
    outlier_count = len(outliers)
    outlier_pct = (outlier_count / len(df)) * 100
    
    outlier_summary.append({
        'Column': col,
        'Outlier_Count': outlier_count,
        'Outlier_Percentage': outlier_pct,
        'Lower_Bound': round(lower_bound, 2),
        'Upper_Bound': round(upper_bound, 2)
    })

# Display outlier summary
outlier_df = pd.DataFrame(outlier_summary)
print("\n📊 Outlier Detection Summary:")
display(outlier_df[outlier_df['Outlier_Count'] > 0])

# Visualize outliers using box plots
if len(numerical_cols) > 0:
    n_cols = min(3, len(numerical_cols))
    n_rows = (len(numerical_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    if n_rows == 1:
        axes = [axes] if n_cols == 1 else axes
    else:
        axes = axes.flatten() if len(numerical_cols) > 1 else [axes]
    
    for i, col in enumerate(numerical_cols[:len(axes)]):
        sns.boxplot(data=df, y=col, ax=axes[i])
        axes[i].set_title(f'Outliers in {col.replace("_", " ").title()}')
        axes[i].grid(True, alpha=0.3)
    
    # Hide extra subplots
    for i in range(len(numerical_cols), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Outlier treatment strategy
print("🔧 OUTLIER TREATMENT STRATEGY")
print("=" * 50)

# Create a copy for outlier treatment
df_cleaned = df.copy()
treatment_log = []

for _, row in outlier_df.iterrows():
    col = row['Column']
    outlier_pct = row['Outlier_Percentage']
    lower_bound = row['Lower_Bound']
    upper_bound = row['Upper_Bound']
    
    if outlier_pct > 0:
        if outlier_pct < 5:  # Conservative approach for < 5% outliers
            # Cap outliers
            original_outliers = ((df_cleaned[col] < lower_bound) | (df_cleaned[col] > upper_bound)).sum()
            df_cleaned[col] = np.where(df_cleaned[col] < lower_bound, lower_bound, df_cleaned[col])
            df_cleaned[col] = np.where(df_cleaned[col] > upper_bound, upper_bound, df_cleaned[col])
            
            treatment_log.append(f"  {col}: CAPPED {original_outliers} outliers ({outlier_pct:.1f}%)")
            
        elif outlier_pct >= 5 and outlier_pct < 15:  # Moderate outliers
            # Use log transformation for right-skewed data
            if df_cleaned[col].min() > 0:  # Can apply log transformation
                df_cleaned[f'{col}_log'] = np.log1p(df_cleaned[col])
                treatment_log.append(f"  {col}: LOG TRANSFORMED (created {col}_log) ({outlier_pct:.1f}%)")
            else:
                # Cap outliers if log transformation not possible
                original_outliers = ((df_cleaned[col] < lower_bound) | (df_cleaned[col] > upper_bound)).sum()
                df_cleaned[col] = np.where(df_cleaned[col] < lower_bound, lower_bound, df_cleaned[col])
                df_cleaned[col] = np.where(df_cleaned[col] > upper_bound, upper_bound, df_cleaned[col])
                treatment_log.append(f"  {col}: CAPPED {original_outliers} outliers ({outlier_pct:.1f}%)")
        else:
            # High percentage of outliers - investigate further
            treatment_log.append(f"  {col}: INVESTIGATE FURTHER - High outlier rate ({outlier_pct:.1f}%)")

if treatment_log:
    print("\n📋 Treatment applied:")
    for log in treatment_log:
        print(log)
else:
    print("✅ No significant outliers requiring treatment!")

print(f"\n📏 Dataset shape after outlier treatment: {df_cleaned.shape}")

## 📊 Data Standardization {#standardization}

In [ ]:
# Data standardization and formatting
print("📊 DATA STANDARDIZATION & FORMATTING")
print("=" * 50)

# Work with cleaned dataset
df_final = df_cleaned.copy()

# Standardize column names (lowercase, replace spaces with underscores)
original_columns = df_final.columns.tolist()
df_final.columns = [col.lower().replace(' ', '_').replace('-', '_') for col in df_final.columns]
new_columns = df_final.columns.tolist()

print("🏷️  Column name standardization:")
for old, new in zip(original_columns, new_columns):
    if old != new:
        print(f"  '{old}' → '{new}'")

# Handle categorical variables
categorical_cols = df_final.select_dtypes(include=['object']).columns.tolist()

if categorical_cols:
    print(f"\n📋 Processing {len(categorical_cols)} categorical columns:")
    
    for col in categorical_cols:
        # Standardize text values
        df_final[col] = df_final[col].astype(str).str.strip().str.lower()
        
        # Show unique values
        unique_vals = df_final[col].value_counts().head(10)
        print(f"\n  {col}:")
        print(f"    Unique values: {df_final[col].nunique()}")
        print(f"    Top values: {list(unique_vals.head(5).index)}")

# Validate numerical columns
numerical_cols_final = df_final.select_dtypes(include=[np.number]).columns.tolist()
print(f"\n🔢 Numerical columns validated: {len(numerical_cols_final)}")
for col in numerical_cols_final:
    print(f"  {col}: {df_final[col].dtype} (Range: {df_final[col].min():.2f} - {df_final[col].max():.2f})")

print(f"\n✅ Data standardization completed!")
print(f"📏 Final dataset shape: {df_final.shape}")

## 🔧 Feature Engineering {#features}

In [ ]:
# Feature engineering for employee retention analysis
print("🔧 FEATURE ENGINEERING")
print("=" * 50)

# Create engineered features based on common HR metrics
features_created = []

# 1. Work-life balance indicators (assuming columns exist)
if 'average_montly_hours' in df_final.columns:
    # Work intensity categories
    df_final['work_intensity'] = pd.cut(df_final['average_montly_hours'], 
                                       bins=[0, 160, 200, 250, float('inf')], 
                                       labels=['normal', 'moderate', 'high', 'extreme'])
    
    # Overtime indicator
    df_final['overtime'] = (df_final['average_montly_hours'] > 200).astype(int)
    
    features_created.extend(['work_intensity', 'overtime'])

# 2. Performance indicators
if 'last_evaluation' in df_final.columns and 'satisfaction_level' in df_final.columns:
    # Performance-satisfaction matrix
    df_final['perf_sat_ratio'] = df_final['last_evaluation'] / (df_final['satisfaction_level'] + 0.01)
    
    # High performer flag
    df_final['high_performer'] = ((df_final['last_evaluation'] > 0.7) & 
                                 (df_final['satisfaction_level'] > 0.6)).astype(int)
    
    features_created.extend(['perf_sat_ratio', 'high_performer'])

# 3. Project workload analysis
if 'number_project' in df_final.columns:
    # Project load categories
    df_final['project_load'] = pd.cut(df_final['number_project'],
                                     bins=[0, 2, 4, 6, float('inf')],
                                     labels=['low', 'normal', 'high', 'extreme'])
    
    features_created.append('project_load')

# 4. Tenure analysis
if 'time_spend_company' in df_final.columns:
    # Tenure categories
    df_final['tenure_category'] = pd.cut(df_final['time_spend_company'],
                                        bins=[0, 2, 5, 8, float('inf')],
                                        labels=['new', 'experienced', 'veteran', 'senior'])
    
    features_created.append('tenure_category')

# 5. Risk scores (combine multiple factors)
risk_factors = []
if 'satisfaction_level' in df_final.columns:
    risk_factors.append('(1 - satisfaction_level)')
if 'overtime' in df_final.columns:
    risk_factors.append('overtime')
if 'last_evaluation' in df_final.columns:
    risk_factors.append('(1 - last_evaluation)')

if risk_factors:
    risk_formula = ' + '.join(risk_factors)
    df_final['attrition_risk_score'] = eval(f"df_final.eval('{risk_formula}')")
    features_created.append('attrition_risk_score')

# Summary of engineered features
if features_created:
    print(f"✅ Created {len(features_created)} new features:")
    for feature in features_created:
        print(f"  • {feature}")
        if df_final[feature].dtype == 'object' or hasattr(df_final[feature], 'cat'):
            print(f"    Values: {df_final[feature].value_counts().head(3).to_dict()}")
        else:
            print(f"    Range: {df_final[feature].min():.3f} - {df_final[feature].max():.3f}")
        print()
else:
    print("ℹ️  No engineered features created (column names may need adjustment)")

print(f"📏 Dataset shape after feature engineering: {df_final.shape}")

## 💾 Final Dataset Export {#export}

In [ ]:
# Final data quality check and export
print("💾 FINAL DATASET PREPARATION & EXPORT")
print("=" * 50)

# Final quality checks
print("🔍 Final Quality Checks:")
print(f"  ✓ Dataset shape: {df_final.shape}")
print(f"  ✓ Missing values: {df_final.isnull().sum().sum()}")
print(f"  ✓ Duplicate records: {df_final.duplicated().sum()}")
print(f"  ✓ Memory usage: {df_final.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Data types summary
print("\n📊 Data Types Summary:")
dtype_summary = df_final.dtypes.value_counts()
for dtype, count in dtype_summary.items():
    print(f"  {dtype}: {count} columns")

# Save cleaned dataset
try:
    # Create processed data directory if it doesn't exist
    import os
    os.makedirs('../data/processed', exist_ok=True)
    
    # Save the cleaned dataset
    output_path = '../data/processed/hr_dataset_cleaned.csv'
    df_final.to_csv(output_path, index=False)
    
    print(f"\n✅ Cleaned dataset saved to: {output_path}")
    print(f"📊 Records saved: {df_final.shape[0]:,}")
    print(f"📋 Features saved: {df_final.shape[1]}")
    
    # Save data dictionary
    data_dict = pd.DataFrame({
        'Column': df_final.columns,
        'Data_Type': [str(dtype) for dtype in df_final.dtypes],
        'Non_Null_Count': [df_final[col].count() for col in df_final.columns],
        'Unique_Values': [df_final[col].nunique() for col in df_final.columns]
    })
    
    dict_path = '../data/processed/data_dictionary.csv'
    data_dict.to_csv(dict_path, index=False)
    print(f"📖 Data dictionary saved to: {dict_path}")
    
except Exception as e:
    print(f"❌ Error saving files: {str(e)}")

# Create summary statistics
print("\n📈 CLEANING SUMMARY STATISTICS")
print("=" * 50)

summary_stats = {
    'Original Records': df_raw.shape[0] if 'df_raw' in locals() else 'N/A',
    'Final Records': df_final.shape[0],
    'Original Features': df_raw.shape[1] if 'df_raw' in locals() else 'N/A',
    'Final Features': df_final.shape[1],
    'Records Removed': df_raw.shape[0] - df_final.shape[0] if 'df_raw' in locals() else 'N/A',
    'Features Added': len(features_created) if features_created else 0,
    'Data Quality Score': f"{((1 - (df_final.isnull().sum().sum() / df_final.size)) * 100):.1f}%"
}

for metric, value in summary_stats.items():
    print(f"  {metric}: {value}")

print("\n" + "="*50)
print("✅ DATA CLEANING & PREPROCESSING COMPLETE!")
print("📋 Ready for modeling phase")
print("🚀 Next step: 03_modeling.ipynb")
print("="*50)

## 📋 Cleaning Summary

### ✅ Completed Tasks:
1. **Data Quality Assessment** - Comprehensive analysis of data structure and quality
2. **Missing Values Treatment** - Handled missing data using appropriate imputation strategies
3. **Duplicate Removal** - Identified and removed duplicate records
4. **Outlier Treatment** - Applied appropriate outlier detection and treatment methods
5. **Data Standardization** - Standardized column names and data formats
6. **Feature Engineering** - Created meaningful derived features for analysis
7. **Data Export** - Saved cleaned dataset for modeling phase

### 📊 Data Quality Improvements:
- Removed inconsistencies and standardized formats
- Applied appropriate data types to all columns
- Created analysis-ready features
- Ensured data integrity for machine learning models

### 🚀 Next Steps:
The cleaned dataset is now ready for the modeling phase where we will:
1. Build predictive models for employee retention
2. Compare different algorithms and approaches
3. Optimize model performance through hyperparameter tuning
4. Validate model results and interpretability

---

**📊 Data Cleaning Complete!**

*Next notebook: `03_modeling.ipynb`*